In [ ]:
#Версия для анализа файлов без возможности использования OCR в скрипте.
#Отсутсвует возможность установки Tesseract и Poppler. 
#Распознование pdf сканов делаем через OCR утилиту.
import fitz  # PyMuPDF
import re
import os

def analyze_pdf_type(pdf_path):
    """Анализирует тип PDF файла"""
    try:
        doc = fitz.open(pdf_path)
        page = doc[0]  # Первая страница

        # Извлекаем текст
        text = page.get_text()

        # Считаем количество символов
        text_length = len(text.strip())

        # Извлекаем изображения
        images = page.get_images()
        image_count = len(images)

        doc.close()

        print(f"📊 Анализ файла: {os.path.basename(pdf_path)}")
        print(f"   Символов текста: {text_length}")
        print(f"   Изображений: {image_count}")

        # Определяем тип
        if text_length < 50 and image_count > 0:
            return "scan", text  # Скан
        elif text_length >= 50:
            return "text", text  # Текстовый PDF
        else:
            return "empty", text  # Пустой/поврежденный

    except Exception as e:
        return "error", str(e)

def extract_data_from_text_pdf(text):
    """Извлекает данные из текстового PDF"""
    try:
        # Извлекаем год
        year_match = re.search(r'(\d{4})\s*г\.', text)
        year = year_match.group(1) if year_match else "Не найден"

        # Извлекаем город
        city = "Не найден"
        city_patterns = [
            r'([А-Я][а-яё]+)\s+\d{4}\s*г\.',
            r'г\.\s*([А-Я][а-яё]+)',
            r'([А-Я][а-яё]+),?\s+\d{4}',
            r'^([А-Я][а-яё]+)\s'
        ]

        for pattern in city_patterns:
            city_match = re.search(pattern, text)
            if city_match:
                city = city_match.group(1)
                if city.lower() not in ['г', 'ул', 'д', 'республика', 'область']:
                    break

        if city == "Не найден":
            city_match = re.search(r'(Иркутск|Москва|Санкт-Петербург|[А-Я][а-яё]{3,})', text)
            if city_match:
                city = city_match.group(1)

        # Извлекаем ВСП №
        vsp_patterns = [
            r'[Оо]фис\s*[№#]?\s*([0-9/\s]+)',
            r'[Вв][Сс][Пп]\s*[№#]?\s*([0-9/\s\-]+)',
            r'(\d{4}/\d\s*\d{2})'
        ]

        vsp = "Не найден"
        for pattern in vsp_patterns:
            vsp_match = re.search(pattern, text)
            if vsp_match:
                vsp = vsp_match.group(1).strip()
                vsp = re.sub(r'\s+', ' ', vsp)
                break

        return {
            'year': year,
            'city': city,
            'vsp': vsp,
            'success': True
        }
    except Exception as e:
        return {
            'error': str(e),
            'success': False
        }

def process_all_pdfs_in_folder(folder_path):
    """Обрабатывает все PDF файлы в папке с определением типа"""

    # Проверяем, существует ли папка
    if not os.path.exists(folder_path):
        print(f"❌ Папка не найдена: {folder_path}")
        return []

    # Получаем список всех PDF файлов в папке
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print("❌ PDF файлы не найдены в папке")
        return []

    print(f"✅ Найдено PDF файлов: {len(pdf_files)}")
    print("=" * 100)

    results = []

    # Обрабатываем каждый PDF файл
    for i, pdf_file in enumerate(pdf_files, 1):
        full_path = os.path.join(folder_path, pdf_file)
        print(f"📄 [{i}/{len(pdf_files)}] Анализ файла: {pdf_file}")

        # Определяем тип PDF
        pdf_type, content = analyze_pdf_type(full_path)

        result = {
            'file': pdf_file,
            'type': pdf_type
        }

        if pdf_type == "text":
            # Текстовый PDF - извлекаем данные
            data = extract_data_from_text_pdf(content)
            result.update(data)
            print(f"   ✅ Текстовый PDF")
            print(f"   🏙️  Город: {data.get('city', 'Не найден')}")
            print(f"   📅  Год: {data.get('year', 'Не найден')}")
            print(f"   🏢  ВСП №: {data.get('vsp', 'Не найден')}")

        elif pdf_type == "scan":
            # Сканированный PDF - сообщаем, что нужен OCR
            result.update({
                'error': 'Сканированный PDF - нужен OCR (Poppler и Tesseract)',
                'success': False
            })
            print(f"   ⚠️  Сканированный PDF (картинка)")
            print(f"   📋 Для обработки нужен OCR:")
            print(f"      1. Установить Poppler и Tesseract")
            print(f"      2. Или загрузить в Google Drive → Google Docs")
            print(f"      3. Или использовать онлайн OCR сервисы")

        elif pdf_type == "empty":
            result.update({
                'error': 'Пустой или поврежденный PDF',
                'success': False
            })
            print(f"   ❌ Пустой или поврежденный PDF")

        else:  # error
            result.update({
                'error': content,
                'success': False
            })
            print(f"   ❌ Ошибка анализа: {content}")

        results.append(result)
        print("-" * 100)

    # Выводим сводную таблицу
    print("📊 СВОДНАЯ ТАБЛИЦА:")
    print("=" * 120)
    print(f"{'№':<3} {'Файл':<35} {'Тип':<15} {'Город':<15} {'Год':<6} {'ВСП №':<15} {'Статус':<20}")
    print("-" * 120)

    for i, result in enumerate(results, 1):
        file_type = result.get('type', 'error')
        status = "✅" if result.get('success') else "❌"

        if file_type == "text" and result.get('success'):
            print(f"{i:<3} {result['file'][:34]:<35} {'Текстовый':<15} {result.get('city', ''):<15} {result.get('year', ''):<6} {result.get('vsp', ''):<15} {status:<20}")
        elif file_type == "scan":
            print(f"{i:<3} {result['file'][:34]:<35} {'Скан':<15} {'':<15} {'':<6} {'':<15} {'⚠️ OCR нужен':<20}")
        else:
            print(f"{i:<3} {result['file'][:34]:<35} {file_type:<15} {'':<15} {'':<6} {'':<15} {'❌ Ошибка':<20}")

    # Статистика
    text_count = sum(1 for r in results if r.get('type') == 'text')
    scan_count = sum(1 for r in results if r.get('type') == 'scan')
    error_count = sum(1 for r in results if r.get('type') in ['error', 'empty'])

    print("\n" + "=" * 100)
    print(f"📈 СТАТИСТИКА:")
    print(f"   Всего файлов: {len(results)}")
    print(f"   Текстовые PDF: {text_count}")
    print(f"   Сканированные PDF: {scan_count}")
    print(f"   С ошибками: {error_count}")

    return results

# Использование
folder_path = r'C:\Users\путь к папке с PDF файлами'
results = process_all_pdfs_in_folder(folder_path)